In [1]:
from sql_metadata import Parser
import pandas as pd
import traceback
from ipywidgets import *
import re

In [2]:
file1= 'etl_wless_hshld_b2b_all_c-develop.hql'
file2='etl_wless_line_b2b_att_po-develop.hql'
file3 = 'etl_wless_line_b2b_In_c-develop.hql'
file4='etl_wless_line_b2b_tmpol-develop.hql'
file5= 'etl_wless_acct_billing_intent_v3-develop.hql'
file6= 'etl_wless_acct_bill_shock_v3-develop.hql'
file7='etl_wless_acct_bill_dspt_late_fee_v2-develop.hql'
file8='etl_wless_multi_acct_bogo_bmsm_dly-develop.hql'
file9='etl_wless_line_play_unl_step_down-develop.hql'
file10='etl_wless_acct_post_ord_call_dflctn-develop.hql'
file11='etl_wless_multi_line_po_carrier-develop.hql'
file12='etl_wless_acct_5g_home_propensity-develop.hql'
file13='etl_wless_multi_acct_aal_v_pplan_rcmnd_dly-develop.hql'
file14='etl_wless_line_accessory_rcmnd_other-develop.hql'
file15='etl_wless_line_accessory_rcmnd_sma-develop.hql'
file16='etl_wless_acct_acct_accsry_blu_wk-develop.hql'
file17='etl_wless_acct_acct_accsry_cas_wk-develop.hql'
file18='etl_wless_acct_acct_accsry_oth_wk-develop.hql'
file19='etl_wless_acct_acct_accsry_pow_wk-develop.hql'
file20='etl_wless_acct_acct_accsry_scr_wk-develop.hql'
file21='etl_wless_acct_acct_accsry_sma_wk-develop.hql'
file22='etl_wless_product_acc_acc_rcmnd-develop.hql'
file23='etl_wless_hshld_b2b_att_p-develop.hql'
file24='etl_wless_acct_buy_voice_dvc_wly-develop.hql'
file25='etl_wless_acct_buy_voice_mfg_wly-develop.hql'
file26='etl_wless_line_call_fltr-develop.hql'
file27='etl_wless_chnl_orch_bshck-develop.hql'



File_CSV='etl_wless_line_b2b_In_c-develop_Model_Fields.csv'

In [3]:
pd.set_option('display.max_rows',500)
pd.options.display.max_colwidth = 500
pd.set_option('display.html.use_mathjax', False)
#pandas.set_option('display.max_colwidth', -1)

In [53]:
import sqlparse
import pandas as pd
from sql_metadata import Parser
from collections.abc import Iterable


def read_hql_file(file_path):
    with open(file_path, 'r') as f:
        raw_hql = f.readlines()
        return raw_hql

def filter_hive_keywords(line):
    if 'set ' in line or 'USE ' in line or 'SET ' in line:
        return ''
    else:
        return line.strip()


def check_alias(parse):
    alias = parse.columns_aliases
    if not alias:
        return False
    else:
        return True
    
def get_table_name(parse):
    try:
        table = parse.tables
        return table[0]
    except:
        return ' . '


def flatten(column_list):
    for col in column_list:
        if isinstance(col, Iterable) and not isinstance(col, (str, bytes)):
            yield from flatten(col)
        else:
            yield col
    
def matching_string(sql):
    l1=[]
    patt=re.compile(r'\),|\)')
    matches=patt.finditer(sql)
    for match in matches:
        #print(match.span())
        a=match.span()
        l1.append(a)
        print(l1)
    b=l1[0][0]
    sql1=sql[0:b]
    print("inmatch")
    return sql1
    
def remove_create(sql):
    try:
        index_slice = []
        create_alias = False
        if 'SELECT' in sql or 'select' in sql:
            split_sql = sql.split('\n')
            for i, line in enumerate(split_sql):
                if 'CREATE' in line:
                    index_slice.append(i)
                if ') AS' in line:
                    if not create_alias:
                        create_alias = True
                        index_slice.append(i)
                if create_alias:
                    del split_sql[index_slice[0]:index_slice[1]+1]
                    #print('DONE: REMOVE CREATE')
                    return '\n'.join(split_sql)
    except Exception as e:
        #print('CANNOT SPLIT: ',e, sql)
        return sql


def get_where(sql):
    is_where = False
    is_group = False
    
    where_clause = ''
    sql_where = sqlparse.format(sql, reindent=True).split('\n')
    #print(sql_where)
    for line in sql_where:
        if '--'  not in line:
            if 'WHERE' in line.upper():
                where_clause = where_clause + ' '.join(line.split(' ')[1:])
                is_where=True
            elif is_where and 'GROUP' not in line.upper():
                where_clause = where_clause + line
            elif is_where and 'Union' in line.upper():
                where_clause = where_clause + line
            if 'GROUP' in line.upper():
                break
            
    main_where=''
    if 'select' in where_clause:
        #print("hello")
        main_where=matching_string(where_clause)
    else:
        main_where=where_clause+main_where
        
        
    print(main_where)        
    return main_where



def get_direct_feature_table(sql):
    try:
        parse = Parser(sql) 
        columns = parse.columns_dict['select']
        columns = list(flatten(columns))
        wheres = get_where(sql)
        #all_columns = columns
        all_columns = [i for n, i in enumerate(columns) if i not in columns[:n]]
        #all_columns = list(set(list(flatten(columns))))
        #print(all_columns)
        table = []
        for entity in all_columns:
            fin_entity = []
            if '.' in entity:
                entities = entity.split('.')
                try:
                    if len(entities) == 3:
                        fin_entity = [entities[0], entities[1], entities[2]]
                        
                    elif len(entities) == 2:
                        fin_entity = ['', entities[0], entities[1]]
                        
                except:
                    fin_entity = [entities[0], entities[1]]
                    
                fin_entity.append(wheres)
                table.append(fin_entity)
            else:
                table_name = get_table_name(parse)
                fin_entity = table_name.split('.') + [entity]
                #print(fin_entity)
                fin_entity.append(wheres)
                table.append(fin_entity)
        return table
    except Exception as e:
        #print("Could not the process the below HQL: ", e)
        #print('HQL :', sql)
        #if 'SELECT' in sql or 'select' in sql:
        #    print("## ", sql, "\n")
        #traceback.print_exc()
        return []  
       
          
           
    
    
    
def process_complex_hql(file_path):
    raw_hql = read_hql_file(file_path)
    #print(raw_hql)
    filtered_query = filter(filter_hive_keywords, raw_hql)
    raw_query = list(filtered_query)
    raw_query = ' '.join(raw_query)
    #print(raw_query)
    raw_query = raw_query.replace('"', "'")
    raw_query = raw_query.replace('${db_inp_data}', 'db_inp_data')
    raw_query = raw_query.replace('${db_dma_data}', 'db_dma_data')
    raw_query = raw_query.replace('${tbl_dma_data}', 'tbl_dma_data')   
    raw_query = raw_query.replace('${db_final_score}', 'db_final_score')
    raw_query = raw_query.replace('${tbl_final_score}', 'tbl_final_score')
    raw_query = raw_query.replace('${db_line_data}', 'db_line_data')
    raw_query = raw_query.replace('${tbl_line_data}', 'tbl_line_data')
    
    multi_hql = raw_query.split(';')
    #return multi_hql
    #print(len(multi_hql))
    table = []
    for query in multi_hql:
        #query = query.replace('"', "'")
        #query_pre = remove_create(query)
        table.extend(get_direct_feature_table(query))
    #return table
    #table_post_process = list(map(lambda x: x if len(x) == 4 else None, table))
    tab_final = [entity for entity in table if entity]
    df = pd.DataFrame(tab_final, columns=['Database', 'Table', 'Feature', 'WHERE'])
    df['WHERE'] = df['WHERE'].str.replace(r'where|WHERE', '')
    dfStyler = df.style.set_properties(**{'text-align': 'left'})
    df=dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    #df=df.style()
    return df



In [56]:
df = process_complex_hql(file23)
df


    

Not supported query type: -- Mandatory default settings
 -- #############################
 -- Optional performance settings
 -- #############################
 -- Optional memory settings
 -- #############################
 -- same as or a small multiple of yarn container size yarn.scheduler.minimum-allocation-mb but never more than yarn.scheduler.maximum-allocation-mb
 -- 80% of hive.tez.container.size
 -- 40% of hive.tez.container.size
 -- 33% of hive.tez.container.size
 -- 10% of hive.tez.container.size
 -- Optional memory settings
 -- #############################
 -- same as or a small multiple of yarn container size yarn.scheduler.minimum-allocation-mb but never more than yarn.scheduler.maximum-allocation-mb
 -- 80% of hive.tez.container.size
 -- 40% of hive.tez.container.size
 -- 33% of hive.tez.container.size
 -- 10% of hive.tez.container.size
 -- getting the household id [crm_hshld_id] column from vzw_ds_prd_sdb_tbls.ds_b2b_line_master
 -- and churn column [target_ind] from vzw_

[(54, 55)]
[(54, 55), (56, 58)]
[(54, 55), (56, 58), (397, 398)]
[(54, 55), (56, 58), (397, 398), (398, 399)]
[(54, 55), (56, 58), (397, 398), (398, 399), (438, 439)]
[(54, 55), (56, 58), (397, 398), (398, 399), (438, 439), (480, 482)]
[(54, 55), (56, 58), (397, 398), (398, 399), (438, 439), (480, 482), (755, 756)]
inmatch
  where rpt_mth = date_format(${run_date},'yyyy-MM-01'

[(48, 49)]
inmatch
  where rank_score_value <= avg_churn_per_hshld 



,Database,Table,Feature,WHERE
0,vzw_uda_prd_allvm,subs_sum_fact_v,cust_id,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
1,vzw_uda_prd_allvm,subs_sum_fact_v,acct_num,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
2,vzw_uda_prd_allvm,subs_sum_fact_v,cust_line_seq_id,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
3,vzw_uda_prd_allvm,subs_sum_fact_v,mtn,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
4,vzw_uda_prd_allvm,subs_sum_fact_v,ecpd_profile_id,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
5,vzw_ds_prdusr_tbls,score_wless_line_b,*,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
6,,b,crm_hshld_id,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
7,,b,owning_area_cd,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
8,,b,ecpd_profile_id,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"
9,vzw_uda_prd_allvm,subs_sum_fact_v,*,"rpt_mth = date_format(${run_date},'yyyy-MM-01'"


In [7]:
#def efficiency(df,df1):
    #print(df)
    #Matched_Features = pd.merge(df, 
                      #df1, 
                      #on=['Database','Table','Feature'], 
                      #how ='inner')
    #print(Matched_Features)
    #efficiency=len(Matched_Features.index)/len(df.index)*100
    #efficiency=print(round(efficiency,2))
    #return round(efficiency,2)

In [8]:
#efficiency(df2,df3[['Database','Table','Feature']])
#df3[["Database","Table","Feature"]]
#df.dtypes

In [9]:
#df_diff = pd.concat([df2,df3[['Database','Table','Feature']]]).drop_duplicates(keep=False)
#df_diff


In [10]:
#df2.compare(df4)
#pd.__version__
#df4.set_index('Feature')


In [11]:
#def sql_process(sql):
    #table[]
#    table=get_direct_feature_table(sql)
#   tab_final = [entity for entity in table if entity]
#    df = pd.DataFrame(tab_final, columns=['Database', 'Table', 'Feature', 'WHERE'])
#    df['WHERE'] = df['WHERE'].str.replace(r'where|WHERE', '')
#    dfStyler = df.style.set_properties(**{'text-align': 'left'})
#    df=dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    #df=df.style()
#    return df


In [6]:
#df=sql_process(sql)
#df
sql='''rpt_mth = date_format(${run_date},'yyyy-MM-01') ), tt2 as (select a.*, b.crm_hshld_id, b.owning_area_cd vzw_ds_prd_sdb_tbls.ds_b2b_line_master b on a.cust_id = b.cust_id and a.acct_num = b.acct_num and a.cust_line_seq_id = b.cust_line_seq_id b.score_mth = last_day(add_months(${run_date}, -1)) and a.rpt_dt =last_day(${run_date}) and a.score_model_id = 'B2B_ATT_PO' ), tt3 as (select a.*, b.ecpd_profile_id from tt2 a left outer join tt1 b on a.cust_id = b.cust_id and a.acct_num = b.acct_num and a.cust_line_seq_id = b.cust_line_seq_id and a.mtn =b.mtn)select *from tt3
'''

In [15]:
#def rege(sql):
#    patt=re.compile('WITH')
#   matches=patt.finditer(sql)
#   
#   for match in matches:
#       print("rrr")
#       print(match)
     
#rege(sql)

In [52]:
a=''''''
if 'select' in sql:
        print("hello")
        a=matching_string(sql)
#matching_string(sql)
print(a)

hello
[(46, 47)]
[(46, 47), (48, 50)]
[(46, 47), (48, 50), (291, 292)]
[(46, 47), (48, 50), (291, 292), (292, 293)]
[(46, 47), (48, 50), (291, 292), (292, 293), (328, 329)]
[(46, 47), (48, 50), (291, 292), (292, 293), (328, 329), (366, 368)]
[(46, 47), (48, 50), (291, 292), (292, 293), (328, 329), (366, 368), (553, 554)]
inmatch
rpt_mth = date_format(${run_date},'yyyy-MM-01'
